# Praktikum Intelligente Sensortechnik 4
Tim Tiedemann, Thomas Lehmann, Tobias De Gasparis

Version 09.01.2024

# Einfache intelligente Sensoren und Datenvorverarbeitung
Im Praktikum 4 geht es um die Verarbeitung hochdimensionaler Daten bzw. das Clustern der Daten.

Lesen Sie sich die Aufgaben gut durch. Sollten Sie eine Aufgabe nicht lösen können, so beschreiben Sie zumindest, wie weit Sie gekommen sind und auf welche Weise Sie vorgegangen sind.

Beachten Sie auf der methodischen Seite, dass die Schritte der Datenerhebung, der Datenauswertung und der Kommentierung des Ergebnisses ausgeführt werden. Alle Diagramme sind korrekt zu beschriften.

Die Aufgaben sind direkt hier als Protokoll zu bearbeiten. Das abgegebene Notebook soll ausführbar sein. Daneben ist der PDF-Export des Notebook mit abzugeben.

**Autoren des Protokolls:** Haron Nazari, Anton Tchekov

# Hintergrund
Intelligente Sensoren sollen nicht nur Daten erfassen, sondern direkt eine Klassifikation mit Informationen auf abstrakterer Ebene aus der Umwelt liefern. Beispielsweise soll eine Geste oder Gegenstände erkannt werden und diese Information über die Schnittstelle zu weiteren Systemkomponenten bereitgestellt werden. In diesem Praktikum wird die Leistungsfähigkeit der Klassifikation einfacher Algorithmen des nicht-überwachten Machine Learnings für diese Aufgabe untersucht.

# Vorbereitungsaufgaben
## Sensoren mit hochdimensionaler Ausgabe von Daten

**Besorgen Sie sich Informationen zu den folgenden Sensoren:**

- Hokuyo URG-04LX-UG01
- Hokuyo UTM-30LX-EW
- Velodyne VLP-16

**Um was für einen Sensor handelt es sich jeweils und welches Messprinzip wird verwendet? Was wird gemessen? Was sind hier der Messbereich, die Auflösung und die Sample-/Messrate?**

**Über welche Schnittstelle(n) können die Messdaten ausgegeben werden? Wie werden jeweils Betriebsspannung und Datenleitungen angeschlossen?**

**URG-04LX**

Der URG-04LX ist ein Laserscanner, welcher einen Bereich scannt. Der Laser
nutzt einen Infrarotsensor und das Sichtfeld beim Scan ist 240 Grad. Er kann
Distanzen bis zu 4000 mm messen und die maximale Fehlerrate beträgt 40 mm bei
4000 mm Entfernung. Das Messprinzip basiert auf der Phasendifferenz.

**Distanz:** Bis zu 4000 mm

**Auflösung:** 1 mm

**Genauigkeit:**

± 1 % des Messwertes bei 1000 - 4000 mm

± 10 mm bei 60 - 1000mm

**Geschwindigkeit:** 100 ms / Scan

**Schnitstellen:** RS-232, USB

**Betriebsspannung:** 5 VDC

**Pinout von Sensor:**

![Pinout von Sensor](urg_04_pinout.png)

**UTM-30LX-EW**

Der UTM-30LX-EW nutzt eine Laserquelle um für jeden Schritt die Distanz zu
messen. Ein Schritt ist 0.25 Grad groß.

**Distanz:** 0.1 - 30 m (60 m Maximum nicht garantiert)

**Auflösung:** 1 mm

**Genauigkeit:**

0.1 - 10m: ± 30 mm

10 - 30 m: ± 50 mm

**Sichtfeld:** 270 Grad

**Schrittweite:** 0.25 Grad

**Schnitstellen:** Ethernet / Synchronous Output

**Geschwindigkeit:** 25 ms

**Velodyne VLP-16**

Der VLP-16 LiDAR Sensor kann über Ethernet (TCP/IPv4) angesteuert werden.
Der Sensor verwendet Time-of-Flight als Messprinzip.

Der Sensor ist ein 3D Sensor mit 100 Meter range

Die Betriebsspannung ist von 8 bis 19 Volt.
Stromverbrauch 0.9 A (8 Watt) in normaler Operation (maximal 3 A).

Die Konfiguration des Sensors kann über ein Web-Inferface erfolgen oder
mit HTTP-Requests an API-Endpunkten.


# Im Labor

## Objektidentifikation in Daten von Laserscannern
Mit LiDARs vom Typ Hokuyo URG-04LX-UG01 sollen Sie die Umgebung der Sensoren erfassen und mittels Clustering-Algorithmus auswerten. Sie sollen untersuchen, ob die Objekte der Umgebung durch den Cluster-Algorithmus gefunden werden.

### Datenerfassung

Verwenden Sie in Absprache mit dem Laborbetreuer einen Sensor vom Typ Hokuyo URG-04LX-UG01 für die Datenerfassung. Wie werden Daten erfasst und wie können sie auf diese zugreifen?

Die Sensoren sind beide an einen Raspberry PI angeschlossen, mit dem man sich über SSH verbindet.

Auf dem Raspberry PI kann man ein Python-Skript ausführen, welches die Daten auf der Konsole
ausgibt. Die Sensoren werden mithilfe der hokuyo-Library über eine Serielle Schittstelle
mit der Baudrate 19200 angesprochen (an /dev/ttyACM0 und /dev/ttyACM1).

Die Daten haben wir dann nach CSV konvertiert.

**Sammeln Sie Daten von einem Scan und versuche Sie die Daten in einer zweidimensionale Karte darzustellen. Welche Form von Koordinatensystem bietet sich hier an und wie können Sie so einen Plot erzeugen? Können Sie Objekte in der Umgebung identifizieren?**

Ein polares Koordinatensystem bietet sich an, da der Sensor Entfernungen in
einem bestimmten Winkel liefert, die von einem zentralen Punkt aus gemessen
wurden.

2D-Karte der Sensordaten von einem Scan:

Erster Sensor (ACM0):

![image](single_scan1_2d.png)

Zweiter Sensor (ACM1):

![image](single_scan2_2d.png)

Wir haben durch "Recherche" herausgefunden in welchem Raum sich die Sensoren befinden.
(Zwischen dem Büro von Silke und 709)

Der Sensor liefert 0 zurück, wenn die Entfernung außerhalb der Reichweite ist oder die
Messung ungültig ist.

Man kann vorne eine unebene Oberfläche erkennen, dabei handelt es sich um ein Fenster mit
Gardinen davor, mit einem Schlitz in der Mitte, sodass der LiDAR durch das Fenster scannt.

Der Raum hat links und rechts eine Tür, welche jeweils von Objekten verdeckt sind und
Lücken in den Daten auslösen.

Links im Raum sind zwei Schränke nebeneinander, mit einer Lücke dazwischen.

Der Raum ist ungefähr 6 Meter lang und 2 Meter breit.

Die LiDARs sind etwa 95 cm voneinander entfernt und 180° zueinander gedreht.

**Python code:**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import csv

title = 'Lidar Messdaten 2D Karte'

dist = np.array(())
angle = np.array(())

with open('ACM1_result.txt', newline='') as csvfile:
	dreader = csv.reader(csvfile, delimiter=',', quotechar='|')
	for row in dreader:
		angle = np.append(angle, np.radians(float(row[0])))
		dist = np.append(dist, float(row[1]))

fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
fig.canvas.manager.set_window_title(title)

ax.plot(angle, dist, '.')
ax.set_rmax(5600)
ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)

ax.set_rticks([0.5, 1, 1.5, 2])
ax.set_rlabel_position(-22.5)
ax.grid(True)

ax.set_title(title, va='bottom')
plt.show()

**Nehmen Sie von zwei LiDARs einen Scan auf. Gehen Sie davon aus, dass einer der Laserscanner im Ursprung eines karthesischen Koordinatensystems (Weltkoordinatensystem) angebracht ist und mit der Mittelachse genau in Richtung der y-Achse ausgerichtet ist. Fügen Sie die Daten des zweiten Laserscanners mittels geeigneter Rotation und Translation (Argumente experimentell bestimmen) in das Weltkoordinatensystem ein, so dass sich ein Gesamtbild in der Karte ergibt. Optional:  Geben Sie die nötige Rotationsmatrix und den Translationsvektor an bzw. bauen Sie Ihre Datentransformation so auf, dass mit einer Rotationsmatrix und einem Translationsvektor gearbeitet wird.**

Da wir aber zwei Sensoren verwenden, ist es sinnvoll, die Werte aus dem polaren
Koordinatensystem in ein kartesisches Koordinatensystem zu  überführen, damit
die Berechnungen für die Translation und Rotation des zweiten Sensors
einfacher sind.

Experimentell bestimmter Translationsvektor:

```
[   0,   910 ]
```

Rotationswinkel:

```
180 °
```

Rotationsmatrix:

```
[
  [  -1,  0 ]
  [   0, -1 ]
]
```

**Gesamtbild 2D-Karte mit Werten beider Sensoren:**

![Zusammengesetzte Daten beider Sensoren](both_scan_2d.png)

**Python code:**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import csv

def pol2cart(rho, phi):
	phi += np.pi / 2
	x = rho * np.cos(phi)
	y = rho * np.sin(phi)
	x = -x
	return(x, y)

sensor2_angle = np.radians(-180)
sensor2_translation = np.array((0.0, 910))
c = np.cos(sensor2_angle)
s = np.sin(sensor2_angle)
sensor2_rotation = np.array(((c, -s), (s, c)))

print("Sensor 2 Translation:")
print(sensor2_translation)
print("Sensor 2 Rotation:")
print(sensor2_rotation)

x1 = np.array(())
y1 = np.array(())
s1_angle = np.array(())
s1_dist = np.array(())
with open('ACM0_result.txt', newline='') as csvfile:
	dreader = csv.reader(csvfile, delimiter=',', quotechar='|')
	for row in dreader:
		a = np.radians(float(row[0]))
		d = float(row[1])
		s1_angle = np.append(s1_angle, a)
		s1_dist = np.append(s1_dist, d)
		(x, y) = pol2cart(d, a)
		x1 = np.append(x1, x)
		y1 = np.append(y1, y)

x2 = np.array(())
y2 = np.array(())
s2_angle = np.array(())
s2_dist = np.array(())
with open('ACM1_result.txt', newline='') as csvfile:
	dreader = csv.reader(csvfile, delimiter=',', quotechar='|')
	for row in dreader:
		a = np.radians(float(row[0]))
		d = float(row[1])
		s2_angle = np.append(s2_angle, a)
		s2_dist = np.append(s2_dist, d)
		(x, y) = pol2cart(d, a)
		out = sensor2_rotation.dot(np.array((x, y)))
		x = out[0]
		y = out[1]
		x += sensor2_translation[0]
		y += sensor2_translation[1]
		x2 = np.append(x2, x)
		y2 = np.append(y2, y)

fig, ax = plt.subplots()
ax.grid(True)

ax.set_aspect('equal', adjustable='box')
plt.xlim(-4000, 4000)
plt.ylim(-4000, 4000)
ax.plot(x1, y1, '.')
ax.plot(x2, y2, '.')
plt.show()

### Clustering der Daten

**Führen Sie nun ein Clustering mittels DBSCAN aus der scikit-learn-Bibliothek auf den gesammelten LiDAR-Daten von einem Sensor durch. Verwenden Sie evtl. erstmal nur einen Teil, z.B. die ersten 100 Sensordimensionen (Entfernungswerte). Finden Sie geeignete Parameter “min_samples” und “eps”. Stellen  Sie die Cluster-Zuordnung der Punkte in der Karte dar.**

Wir haben als `eps` 180 gewählt und für `min_samples` 4.

Clusterzuordnung von DBSCAN:

![DBSCAN Ergebnisse](dbscan.png)

**Wieviele Cluster werden in Ihrem Datensatz identifiziert? Welche Objekte könnten es sein?**

Es werden insgesamt 8 Cluster identifiziert. Es werden die Wände, beide Gardinen und der
Schrank als eigener Cluster erkannt.

**Sollte im LiDAR-Datensatz eine Standardisierung durchgeführt werden? Was spricht dafür und was dagegen? Beachten Sie die Arbeitsweise von DBSCAN.**

Es ist eher nicht sinnvoll, eine Standardiserung durchzuführen,
da alle Entfernungsmesswerte die gleiche Einheit haben, und die absolute
Entfernungsinformation (z.B. in Metern) wichtig ist und bei der
Standartisierung verloren gehen würde. Zudem ist DBSCAN resistent gegenüber
Ausreißern, da es dichtebasiert arbeitet, weswegen eine Standardisierung
nicht notwendig wäre.

**Können Sie eine Daumenregel für die Wahl der Parameter von DBSCAN ableiten, so dass Objekte in der Umgebung als getrennte Objekte erkannt werden?**

DBSCAN benötigt zwei Eingabeparameter, die minimum samples (`min_samples`) und
Epsilon (`eps`).

Für `min_samples` kann man als Faustregel `2 * dim` wählen, wobei `dim` die Anzahl
der Dimensionen des Datensatzes ist. Dabei sollte `min_samples` mindestens `3` sein.
Je größer der Datensatz ist, desto größer sollte man `min_samples` wählen.

`eps` kann man mithilfe eines k-distance-graph bestimmen, indem man
`k = min_samples` setzt und `eps` auf den Wert setzt, bei dem der Graph einen
starken Knick zeigt.

## Bewegungserkennung
Aus den Daten der Sensoren des Nucleo-Moduls soll nun eine Bewegung identifiziert werden.

### Datenerfassung

Verwenden Sie im folgenden das IKS01A3-Board und Ihr Programm, wie Sie es zum Sammeln der 12-dimensionalen 1024-Sample Datensätzen verwendet haben.

Nehmen Sie einen Datensatz auf, in dem das Modul zunächst still in einer Postion (und Ausrichtung) steht, dann eine Bewegung durchgeführt wird (z.B. Translation und dann Rotation) und das Modul dann still in einer anderen Position (und Ausrichtung) steht (alles innerhalb eines 1024-Sample-Datensatzes). Dokumentieren (Skizze) Sie die Bewegung! (Empfehlung: Filmen Sie die Bewegung, damit Sie ggf. einige Punkte in den Datensätzen bei der Auswertung besser einer Bewegung zuordnen können.)

Optional können Sie versuchen, zusätzlich einen Datensatz mit drei Ruhephasen innerhalb der 1024 Samples aufzunehmen.

### Auswertung

**Stellen Sie sich diesen Datensatz graphisch als Plot (2D/3D) dar und prüfen Sie, ob die verschiedenen Phasen zu erkennen sind. Zeigen Sie hier einen geeigneten Plot/Ausschnitt.**

**TODO**: Bild erzeugen und plotten

Plots des Datensatzes:

![Board Bewegung](imuplot.png)

**Versuchen Sie mittels Clustering-Algorithmus (k-Means und DBSCAN) die Bewegungen/Bewegungsabschnitte in den Daten zu identifizieren. Welches Verfahren kann wie gut die Bewegung/Bewegungsabschnitte bei welchen Parametern identifizieren? Welche Normierung/Skalierung war sinnvoll? Ist eine Reduktion mit PCA sinnvoll? Stellen Sie die Ergebnisse des Clusterings ggf. auch in Plots über die Zeit dar.**

**Fügen Sie hier Ihre Analyse kommentiert ein.**

TODO

### Sensorsystem

**Wenn Sie nun das Sensorboard zusammen mit den Cluster-Algorithmen als Sensorsystem betrachten, welche Information(en) könnte die Schnittstelle des Sensorsystems als Ergebnis des Clustern bereitstellen? 
Was wäre bei verschiedenen Bewegungen (2 Ruhephasen, 3 Ruhephasen) eine mögliche Ausgabe?**

Die Schnittstelle könnte einen Zeitabschnitt zurückliefern, der mit `t_start`
und `t_end` definiert ist, zusammen mit einer Beschreibung, was in diesem
Zeitabschnitt für eine Bewegung stattgefunden hat.

z.B.:

| t_start | t_end | Art der Bewegung                 |
|---------|-------|----------------------------------|
|    0.00 |  1.00 | Ruhephase                        |
|    1.00 |  1.50 | Bewegung entlang +X und -Z Achse |
|    1.50 |  2.00 | Drehung um Y Achse               |
|    2.00 |  3.00 | Ruhephase                        |
